# Libraries

In [113]:
import pandas as pd
import numpy as np
import requests
import re

from bs4 import BeautifulSoup
from datetime import datetime

# ID, name, category, price and datetime

In [114]:
url02 = "https://www2.hm.com/en_us/men/products/jeans.html?sort=stock&image-size=small&image=model&offset=0&page-size=72"

# conteúdo de headers é padrão
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url02, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

products = soup.find('ul', 'products-listing small')
products_list = products.find_all('article', 'hm-product-item')
products_id = [p.get('data-articlecode') for p in products_list]
products_category = [p.get('data-category') for p in products_list]

product_name = products.find_all('a', 'link')
product_name = [p.get_text() for p in product_name]

product_price = products.find_all('span', 'price regular')
product_price = [p.get_text() for p in product_price]

data = pd.DataFrame([products_id, product_name, products_category, product_price]).T
data.columns = ['id', 'product_name', 'product_type', 'price']

datetime.now()
datetime.now().strftime('%Y-%m-%d %H:%M:%S')
data['datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

KeyboardInterrupt: 

In [ ]:
data

# Color, Fit, composition, more sustainable materials and size 

In [ ]:
total_itens = soup.find_all('h2', 'load-more-heading')[0].get('data-total')
total_itens

In [ ]:
pagination_number = np.round(int(total_itens)/36)
pagination_number

In [ ]:
soup.find_all('body', 'pdp-page not-signed-in using-mouse')

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

#empty dataframe
df_final = pd.DataFrame()

# All columns found on website
cols = ['Art. No.', 'Composition', 'Fit', 'More sustainable materials', 'Size']
df_pattern = pd.DataFrame(columns=cols)

for i in range(len(data)):

    #API request
    # conteúdo de headers é padrão
    url02 = "https://www2.hm.com/en_us/productpage." + data.loc[i, 'id'] + ".html"+ "?page-size=" + str(int(pagination_number*36))

    page = requests.get(url02, headers=headers)

    #Beautiful Soup object
    soup = BeautifulSoup(page.text, 'html.parser')

    # ============================= Color =========================

    #product list
    product_list = soup.find_all('a', 'filter-option miniature')

    #color
    product_color = [p.get('data-color') for p in product_list] 

    #id
    product_id = [p.get('data-articlecode') for p in product_list]

    #dataframe
    df_color = pd.DataFrame([product_id, product_color]).T
    df_color.columns = ['id', 'color']

    #generate style id + color id
    df_color['style_id'] = df_color['id'].apply(lambda x: x[:-3])
    df_color['color_id'] = df_color['id'].apply(lambda x: x[-3:])

    # ============================ Composition =====================

    # Product list
    product_composition_list = soup.find_all('div', 'pdp-description-list-item')

    # Composition
    product_composition = [list( filter( None, p.get_text().split('\n') ) ) for p in product_composition_list]

  
    # dataframe
    df_composition = pd.DataFrame(product_composition).T

    # Columns name
    print(df_composition.iloc[0])
    df_composition.columns = df_composition.iloc[0]

    # Filling None/NA values
    df_composition = df_composition.iloc[1:].fillna(method='ffill')

    # The same number of columns (pattern)
    df_composition = pd.concat( [df_pattern, df_composition] )
    
    # Generate Style ID + Color ID
    # All values, but the last three values
    df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
    df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])
    
    # ======================= Merging color + composition ==========================
    data_merge = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition', 'More sustainable materials', 'Size']], how='left', on='style_id')

    # ======================= Concatenate ==========================================
    df_final = pd.concat( [df_final, data_merge], axis=0 )

In [ ]:
df_composition

In [ ]:
# Creating style_id + color_id
data['style_id'] = data['id'].apply(lambda x: x[:-3])
data['color_id'] = data['id'].apply(lambda x: x[-3:])

data_raw = pd.merge( data, df_final[['color', 'style_id', 'Fit', 'Composition', 'More sustainable materials', 'Size']], how='left', on='style_id')

In [ ]:
data_raw

# Teste cor preta

In [ ]:
soup.find_all('a', role='radio')

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

#empty dataframe
df_final = pd.DataFrame()

# All columns found on website
cols = ['Art. No.', 'Composition', 'Fit', 'More sustainable materials', 'Size']
df_pattern = pd.DataFrame(columns=cols)

for i in range(len(data)):

    #API request
    # conteúdo de headers é padrão
    url02 = "https://www2.hm.com/en_us/productpage." + data.loc[i, 'id'] + ".html"+ "?page-size=" + str(int(pagination_number*36))

    page = requests.get(url02, headers=headers)

    #Beautiful Soup object
    soup = BeautifulSoup(page.text, 'html.parser')

    # ============================= Color =========================

    #product list
    product_list = soup.find_all('a', role='radio')

    #color
    product_color = [p.get('data-color') for p in product_list] 

    #id
    product_id = [p.get('data-articlecode') for p in product_list]

    #dataframe
    df_color = pd.DataFrame([product_id, product_color]).T
    df_color.columns = ['id', 'color']

    #generate style id + color id
    df_color['style_id'] = df_color['id'].apply(lambda x: x[:-3])
    df_color['color_id'] = df_color['id'].apply(lambda x: x[-3:])

    # ============================ Composition =====================

    # Product list
    product_composition_list = soup.find_all('div', 'pdp-description-list-item')

    # Composition
    product_composition = [list( filter( None, p.get_text().split('\n') ) ) for p in product_composition_list]

  
    # dataframe
    df_composition = pd.DataFrame(product_composition).T

    # Columns name
    df_composition.columns = df_composition.iloc[0]

    # Filling None/NA values
    df_composition = df_composition.iloc[1:].fillna(method='ffill')

    # The same number of columns (pattern)
    df_composition = pd.concat( [df_pattern, df_composition] )
    
    # Generate Style ID + Color ID
    # All values, but the last three values
    df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
    df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])
    
    # ======================= Merging color + composition ==========================
    data_merge = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition', 'More sustainable materials', 'Size']], how='left', on='style_id')

    # ======================= Concatenate ==========================================
    df_final = pd.concat( [df_final, data_merge], axis=0 )

In [ ]:
# Creating style_id + color_id
data['style_id'] = data['id'].apply(lambda x: x[:-3])
data['color_id'] = data['id'].apply(lambda x: x[-3:])

data_raw = pd.merge( data, df_final[['color', 'style_id', 'Fit', 'Composition', 'More sustainable materials', 'Size']], how='left', on='style_id')

In [ ]:
data_raw

In [ ]:
#data_raw.to_csv('data_raw.csv')

# Tratando os dados

In [615]:
url = "https://raw.githubusercontent.com/lucasquemelli/ds_ao_dev/main/data_raw.csv"
data = pd.read_csv(url)

In [616]:
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition,More sustainable materials,Size
0,0,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
1,1,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
2,2,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
3,3,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
4,4,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"


#### Test to identify Na values

In [617]:
data['price'].isna().sum()

0

#### In case of no Na values

In [618]:
data['price'].apply(lambda x: x.replace('$ ', ''))

0       19.99
1       19.99
2       19.99
3       19.99
4       19.99
        ...  
3652    29.99
3653    29.99
3654    29.99
3655    29.99
3656    29.99
Name: price, Length: 3657, dtype: object

#### In case of Na values

In [619]:
data['price'].apply(lambda x: x.replace('$ ', '') if pd.notnull(x) else x)

0       19.99
1       19.99
2       19.99
3       19.99
4       19.99
        ...  
3652    29.99
3653    29.99
3654    29.99
3655    29.99
3656    29.99
Name: price, Length: 3657, dtype: object

#### Another manner

In [620]:
data.apply(lambda x: x['price'].replace('$ ', '') if pd.notnull(x['price']) else x['price'], axis = 1)

0       19.99
1       19.99
2       19.99
3       19.99
4       19.99
        ...  
3652    29.99
3653    29.99
3654    29.99
3655    29.99
3656    29.99
Length: 3657, dtype: object

#### Only remove dollar sign if the color is gray (both conditional)

In [621]:
data['color'].unique()

array(['Black', 'Light denim blue', 'Denim blue', 'Dark blue',
       'Dark denim blue', 'Dark gray', 'Light denim blue/trashed',
       'Black/washed', 'Black washed out', 'White',
       'Dark denim blue/trashed', 'Black/trashed', 'Dark blue/Trashed',
       'Dark blue/washed out', 'Denim blue/trashed', 'Denim gray',
       'Light denim gray/Trashed', 'Gray', 'Pale denim blue',
       'Light gray', 'Black/No fade black', 'Light blue', 'Midnight blue',
       'Light denim blue/Smiley®', 'Graphite gray', 'Black denim',
       'Blue washed out', 'White denim', 'Light blue washed out',
       'Super light denim blue', 'Denim blue/bleached', 'Black/bleached',
       'Black/washed out', 'Dark blue denim', 'Denim blue washed out',
       'Gray denim', 'Black washed-out', 'Blue'], dtype=object)

In [622]:
data[['price', 'color']].apply(lambda x: x['price'].replace('$ ', '')
                                   if (pd.notnull(x['price'])) & (x['color']=='Gray')
                                   else x['price'], axis=1)

0       $ 19.99
1       $ 19.99
2       $ 19.99
3       $ 19.99
4       $ 19.99
         ...   
3652    $ 29.99
3653    $ 29.99
3654    $ 29.99
3655      29.99
3656    $ 29.99
Length: 3657, dtype: object

In [623]:
data.tail(3)

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition,More sustainable materials,Size
3654,3654,865734001,Relaxed Tapered Pull-on Jeans,men_jeans_relaxed,$ 29.99,2022-01-24 14:33:19,865734,1,Light denim blue,Relaxed fit,Cotton 100%,NaN,NaN
3655,3655,865734001,Relaxed Tapered Pull-on Jeans,men_jeans_relaxed,$ 29.99,2022-01-24 14:33:19,865734,1,Gray,Relaxed fit,Cotton 100%,NaN,NaN
3656,3656,865734001,Relaxed Tapered Pull-on Jeans,men_jeans_relaxed,$ 29.99,2022-01-24 14:33:19,865734,1,Pale denim blue,Relaxed fit,Cotton 100%,NaN,NaN


# Limpeza dos dados 

In [624]:
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition,More sustainable materials,Size
0,0,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
1,1,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
2,2,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
3,3,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
4,4,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"


#### data types

In [625]:
data.dtypes

Unnamed: 0                     int64
id                             int64
product_name                  object
product_type                  object
price                         object
datetime                      object
style_id                       int64
color_id                       int64
color                         object
Fit                           object
Composition                   object
More sustainable materials    object
Size                          object
dtype: object

#### Limpeza e tratamento de dados das colunas

In [626]:
data.isna().sum()

Unnamed: 0                       0
id                               0
product_name                     0
product_type                     0
price                            0
datetime                         0
style_id                         0
color_id                         0
color                            0
Fit                              0
Composition                      0
More sustainable materials    2448
Size                          1152
dtype: int64

Poderíamos deletar os valores de NA da seguinte forma:

In [627]:
data.dropna(subset=['Size'])

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition,More sustainable materials,Size
0,0,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
1,1,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
2,2,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
3,3,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
4,4,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",NaN,"The model is 185cm/6'1"" and wears a size 31/32"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,3647,993887002,Hybrid Regular Denim Joggers,men_jeans_regular,$ 44.99,2022-01-24 14:33:19,993887,2,Denim blue,Regular fit,"Cotton 77%, Polyester 21%, Spandex 2%",Recycled cotton 20%,"The model is 189cm/6'2"" and wears a size 32/32"
3648,3648,993887002,Hybrid Regular Denim Joggers,men_jeans_regular,$ 44.99,2022-01-24 14:33:19,993887,2,Denim blue,Regular fit,"Cotton 77%, Polyester 21%, Spandex 2%",Recycled polyester 21%,"The model is 189cm/6'2"" and wears a size 32/32"
3649,3649,993887002,Hybrid Regular Denim Joggers,men_jeans_regular,$ 44.99,2022-01-24 14:33:19,993887,2,Denim blue,Regular fit,"Cotton 77%, Polyester 21%, Spandex 2%",Recycled cotton 20%,"The model is 189cm/6'2"" and wears a size 32/32"
3650,3650,993887002,Hybrid Regular Denim Joggers,men_jeans_regular,$ 44.99,2022-01-24 14:33:19,993887,2,Black,Regular fit,"Cotton 77%, Polyester 21%, Spandex 2%",Recycled polyester 21%,"The model is 189cm/6'2"" and wears a size 32/32"


Contudo, ao deletar os valores de NA, estaríamos deletando os dados das outras colunas que estivessem nas mesmas linhas dos valores NA. Por isso, escolhemos deletar as colunas 'More sustainable materials' e 'Size' porque contêm uma quantidade expressiva de NAs e não são necessárias para a análise - de acordo com o planejamento. 

Vamos, também, criar o dataframe df para termos um dataframe alternativo com as colunas 'More sustainable materials' e 'Size'.

In [628]:
df = data.copy()

data = data.drop(['More sustainable materials', 'Size'], axis=1)

In [629]:
df = df.dropna(subset=['More sustainable materials', 'Size'])
df.isna().sum()

Unnamed: 0                    0
id                            0
product_name                  0
product_type                  0
price                         0
datetime                      0
style_id                      0
color_id                      0
color                         0
Fit                           0
Composition                   0
More sustainable materials    0
Size                          0
dtype: int64

In [630]:
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition
0,0,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
1,1,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Shell: Cotton 99%, Spandex 1%"
2,2,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
3,3,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%"
4,4,1024256001,Slim Jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"


Agora vamos colocar o conteúdo de product_name em lowercase e separado por underscore:

In [631]:
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower() )
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition
0,0,1024256001,slim_jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
1,1,1024256001,slim_jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Shell: Cotton 99%, Spandex 1%"
2,2,1024256001,slim_jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
3,3,1024256001,slim_jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%"
4,4,1024256001,slim_jeans,men_jeans_slim,$ 19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"


Removendo cifrão e mudando o tipo para float:

In [632]:
data['price'] = data['price'].apply(lambda x: x.replace('$ ', '')).astype(float)
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition
0,0,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
1,1,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Shell: Cotton 99%, Spandex 1%"
2,2,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
3,3,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%"
4,4,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"


Convertendo para datetime:

In [633]:
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d %H:%M:%S')
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition
0,0,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
1,1,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,Black,Slim fit,"Shell: Cotton 99%, Spandex 1%"
2,2,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
3,3,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%"
4,4,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"


Colocando underscore nos espaços e transformando todos os caracteres em lowercase: 

In [634]:
data['color'] = data['color'].apply(lambda x: x.replace(' ', '_').replace('/', '_').replace('-', '_').lower() )
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition
0,0,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,black,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
1,1,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,black,Slim fit,"Shell: Cotton 99%, Spandex 1%"
2,2,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,light_denim_blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
3,3,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,light_denim_blue,Slim fit,"Shell: Cotton 99%, Spandex 1%"
4,4,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,light_denim_blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"


In [635]:
data['color'].unique()

array(['black', 'light_denim_blue', 'denim_blue', 'dark_blue',
       'dark_denim_blue', 'dark_gray', 'light_denim_blue_trashed',
       'black_washed', 'black_washed_out', 'white',
       'dark_denim_blue_trashed', 'black_trashed', 'dark_blue_trashed',
       'dark_blue_washed_out', 'denim_blue_trashed', 'denim_gray',
       'light_denim_gray_trashed', 'gray', 'pale_denim_blue',
       'light_gray', 'black_no_fade_black', 'light_blue', 'midnight_blue',
       'light_denim_blue_smiley®', 'graphite_gray', 'black_denim',
       'blue_washed_out', 'white_denim', 'light_blue_washed_out',
       'super_light_denim_blue', 'denim_blue_bleached', 'black_bleached',
       'dark_blue_denim', 'denim_blue_washed_out', 'gray_denim', 'blue'],
      dtype=object)

In [636]:
data['Fit'] = data['Fit'].apply(lambda x: x.replace(' ', '_').lower())
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition
0,0,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,black,slim_fit,"Pocket lining: Polyester 65%, Cotton 35%"
1,1,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,black,slim_fit,"Shell: Cotton 99%, Spandex 1%"
2,2,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,light_denim_blue,slim_fit,"Pocket lining: Polyester 65%, Cotton 35%"
3,3,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,light_denim_blue,slim_fit,"Shell: Cotton 99%, Spandex 1%"
4,4,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,light_denim_blue,slim_fit,"Pocket lining: Polyester 65%, Cotton 35%"


In [637]:
data['Fit'].unique()

array(['slim_fit', 'skinny_fit', 'relaxed_fit', 'regular_fit',
       'loose_fit'], dtype=object)

#### Coluna Size

In [638]:
df['Size'].unique()

array(['The model is 187cm/6\'2" and wears a size 32/32',
       'The model is 187cm/6\'2" and wears a size 31/32',
       'The model is 189cm/6\'2" and wears a size 32/32',
       'The model is 185cm/6\'1" and wears a size 31/32',
       'The model is 187cm/6\'2" and wears a size 32',
       'The model is 188cm/6\'2" and wears a size M',
       'The model is 187cm/6\'2" and wears a size M',
       'The model is 188cm/6\'2" and wears a size 31/30',
       'The model is 186cm/6\'1" and wears a size 31',
       'The model is 186cm/6\'1" and wears a size 31/32'], dtype=object)

Vamos criar uma coluna para o tamanho do modelo:

In [639]:
import re

df['model_size'] = df['Size'].apply(lambda x: re.search('\d{3}cm', x).group(0))
df.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition,More sustainable materials,Size,model_size
168,168,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Black,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187cm
169,169,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Black,Skinny fit,Pocket lining: Cotton 100%,Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187cm
170,170,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Denim blue,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187cm
171,171,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Denim blue,Skinny fit,Pocket lining: Cotton 100%,Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187cm
172,172,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Dark gray,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187cm


In [640]:
df['model_size'] = df['Size'].apply(lambda x: re.search('\d+', x).group(0))
df.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition,More sustainable materials,Size,model_size
168,168,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Black,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187
169,169,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Black,Skinny fit,Pocket lining: Cotton 100%,Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187
170,170,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Denim blue,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187
171,171,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Denim blue,Skinny fit,Pocket lining: Cotton 100%,Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187
172,172,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Dark gray,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187


Agora vamos criar uma coluna para o tamanho da calça:

In [641]:
df['jeans_size'] = df['Size'].str.extract('(\d+/\\d+)')
df.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition,More sustainable materials,Size,model_size,jeans_size
168,168,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Black,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187,32/32
169,169,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Black,Skinny fit,Pocket lining: Cotton 100%,Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187,32/32
170,170,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Denim blue,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187,32/32
171,171,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Denim blue,Skinny fit,Pocket lining: Cotton 100%,Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187,32/32
172,172,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Dark gray,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,"The model is 187cm/6'2"" and wears a size 32/32",187,32/32


In [642]:
df['jeans_size'].unique()

array(['32/32', '31/32', nan, '31/30'], dtype=object)

In [643]:
df = df.drop(columns=['Size'], axis=1)
df.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition,More sustainable materials,model_size,jeans_size
168,168,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Black,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,187,32/32
169,169,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Black,Skinny fit,Pocket lining: Cotton 100%,Recycled cotton 20%,187,32/32
170,170,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Denim blue,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,187,32/32
171,171,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Denim blue,Skinny fit,Pocket lining: Cotton 100%,Recycled cotton 20%,187,32/32
172,172,985159001,Skinny Jeans,men_jeans_skinny,$ 24.99,2022-01-24 14:33:19,985159,1,Dark gray,Skinny fit,"Shell: Cotton 99%, Spandex 1%",Recycled cotton 20%,187,32/32


#### Composition Column

In [644]:
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition
0,0,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,black,slim_fit,"Pocket lining: Polyester 65%, Cotton 35%"
1,1,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,black,slim_fit,"Shell: Cotton 99%, Spandex 1%"
2,2,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,light_denim_blue,slim_fit,"Pocket lining: Polyester 65%, Cotton 35%"
3,3,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,light_denim_blue,slim_fit,"Shell: Cotton 99%, Spandex 1%"
4,4,1024256001,slim_jeans,men_jeans_slim,19.99,2022-01-24 14:33:19,1024256,1,light_denim_blue,slim_fit,"Pocket lining: Polyester 65%, Cotton 35%"


In [645]:
data['Composition'].unique()

array(['Pocket lining: Polyester 65%, Cotton 35%',
       'Shell: Cotton 99%, Spandex 1%', 'Pocket lining: Cotton 100%',
       'Cotton 98%, Spandex 2%', 'Lining: Polyester 100%',
       'Shell: Cotton 100%', 'Cotton 99%, Spandex 1%',
       'Cotton 77%, Polyester 21%, Spandex 2%',
       'Cotton 79%, Polyester 20%, Spandex 1%',
       'Cotton 80%, Polyester 19%, Spandex 1%',
       'Cotton 78%, Polyester 21%, Spandex 1%',
       'Cotton 79%, Polyester 19%, Spandex 2%',
       'Shell: Cotton 90%, Elasterell-P 8%, Spandex 2%',
       'Pocket lining: Polyester 80%, Cotton 20%',
       'Shell: Cotton 89%, Elasterell-P 9%, Spandex 2%',
       'Shell: Cotton 98%, Spandex 2%', 'Cotton 100%',
       'Pocket: Cotton 100%', 'Cotton 93%, Polyester 6%, Spandex 1%'],
      dtype=object)

Vamos remover todas as linhas que contenham os termos 'Pocket', 'Pocket lining', 'Lining' e 'Shell'. Essa escolha foi tomada sabendo que o que é mais importante no momento é o material do jeans, não o material do bolso ou do forro. 

Pocket: bolso

Lining: forro

Pocket lining: forro do bolso

Shell: tecido principal para o forro do jeans

In [646]:
data = data[~data['Composition'].str.contains('Pocket lining:')]

In [647]:
data = data[~data['Composition'].str.contains('Lining:')]

In [648]:
data = data[~data['Composition'].str.contains('Shell:')]

In [649]:
data = data[~data['Composition'].str.contains('Pocket:')]

In [650]:
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition
280,280,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue_trashed,skinny_fit,"Cotton 98%, Spandex 2%"
282,282,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,denim_blue,skinny_fit,"Cotton 98%, Spandex 2%"
284,284,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed,skinny_fit,"Cotton 98%, Spandex 2%"
286,286,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue,skinny_fit,"Cotton 98%, Spandex 2%"
288,288,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed_out,skinny_fit,"Cotton 98%, Spandex 2%"


In [651]:
data['Composition'].unique()

array(['Cotton 98%, Spandex 2%', 'Cotton 99%, Spandex 1%',
       'Cotton 77%, Polyester 21%, Spandex 2%',
       'Cotton 79%, Polyester 20%, Spandex 1%',
       'Cotton 80%, Polyester 19%, Spandex 1%',
       'Cotton 78%, Polyester 21%, Spandex 1%',
       'Cotton 79%, Polyester 19%, Spandex 2%', 'Cotton 100%',
       'Cotton 93%, Polyester 6%, Spandex 1%'], dtype=object)

Quebrar a 'Composition' por colunas diferentes de acordo com o material:

In [652]:
df1 = data['Composition'].str.split(',', expand=True)
df1

,0,1,2
280,Cotton 98%,Spandex 2%,None
282,Cotton 98%,Spandex 2%,None
284,Cotton 98%,Spandex 2%,None
286,Cotton 98%,Spandex 2%,None
288,Cotton 98%,Spandex 2%,None
...,...,...,...
3652,Cotton 100%,None,None
3653,Cotton 100%,None,None
3654,Cotton 100%,None,None
3655,Cotton 100%,None,None


In [653]:
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition
280,280,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue_trashed,skinny_fit,"Cotton 98%, Spandex 2%"
282,282,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,denim_blue,skinny_fit,"Cotton 98%, Spandex 2%"
284,284,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed,skinny_fit,"Cotton 98%, Spandex 2%"
286,286,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue,skinny_fit,"Cotton 98%, Spandex 2%"
288,288,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed_out,skinny_fit,"Cotton 98%, Spandex 2%"


Conteúdo de cada coluna:

In [654]:
df1[0].unique()

array(['Cotton 98%', 'Cotton 99%', 'Cotton 77%', 'Cotton 79%',
       'Cotton 80%', 'Cotton 78%', 'Cotton 100%', 'Cotton 93%'],
      dtype=object)

In [655]:
df1[1].unique()

array([' Spandex 2%', ' Spandex 1%', ' Polyester 21%', ' Polyester 20%',
       ' Polyester 19%', None, ' Polyester 6%'], dtype=object)

In [656]:
df1[2].unique()

array([None, ' Spandex 2%', ' Spandex 1%'], dtype=object)

Para separar o conteúdo das colunas acima, primeiro definimos a ordem que desejamos:

Cotton | Polyester | Spandex

In [657]:
df_ref = pd.DataFrame(index=np.arange(len(data)), columns=['Cotton', 'Polyester', 'Spandex'])

Devemos criar um dataframe com a mesma quantidade de linhas do dataframe com o qual estamos trabalhando - que neste caso é o *data* - se quisermos juntá-los. Por isso, o *df_ref* contém o termo **np.arange(len(data))**. Ainda assim, devemos verificar:

In [658]:
data.shape

(1037, 11)

In [659]:
df_ref.shape

(1037, 3)

Coluna Cotton

In [660]:
df_cotton = df1[0]

In [661]:
df_cotton.shape

(1037,)

In [662]:
df_cotton = df1[0]
df_cotton.name = 'cotton'

df_ref = pd.concat([df_ref.reset_index(), df_cotton.reset_index()], axis=1)
df_ref

,index,Cotton,Polyester,Spandex,index,cotton
0,0,NaN,NaN,NaN,280,Cotton 98%
1,1,NaN,NaN,NaN,282,Cotton 98%
2,2,NaN,NaN,NaN,284,Cotton 98%
3,3,NaN,NaN,NaN,286,Cotton 98%
4,4,NaN,NaN,NaN,288,Cotton 98%
...,...,...,...,...,...,...
1032,1032,NaN,NaN,NaN,3652,Cotton 100%
1033,1033,NaN,NaN,NaN,3653,Cotton 100%
1034,1034,NaN,NaN,NaN,3654,Cotton 100%
1035,1035,NaN,NaN,NaN,3655,Cotton 100%


Para mantermos a última coluna de index, deveríamos usar: **duplicated(keep='last')**. Mas como vamos manter a primeira, faremos:

In [663]:
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated()]
df_ref.head()

,index,Cotton,Polyester,Spandex,cotton
0,0,NaN,NaN,NaN,Cotton 98%
1,1,NaN,NaN,NaN,Cotton 98%
2,2,NaN,NaN,NaN,Cotton 98%
3,3,NaN,NaN,NaN,Cotton 98%
4,4,NaN,NaN,NaN,Cotton 98%


In [664]:
df_ref = df_ref.drop(columns=['Cotton'], axis=1)
df_ref.head()

,index,Polyester,Spandex,cotton
0,0,NaN,NaN,Cotton 98%
1,1,NaN,NaN,Cotton 98%
2,2,NaN,NaN,Cotton 98%
3,3,NaN,NaN,Cotton 98%
4,4,NaN,NaN,Cotton 98%


Coluna Polyester

In [665]:
df1[1].unique()

array([' Spandex 2%', ' Spandex 1%', ' Polyester 21%', ' Polyester 20%',
       ' Polyester 19%', None, ' Polyester 6%'], dtype=object)

In [666]:
df_polyester = df1.loc[df1[1].str.contains('Polyester', na=True), 1]
df_polyester.name = 'polyester'

In [667]:
df_ref = pd.concat([df_ref.reset_index(), df_polyester.reset_index()], axis=1)
df_ref = df_ref.drop(columns=['Polyester'], axis=1)
df_ref.head()

,level_0,index,Spandex,cotton,index,polyester
0,0,0,NaN,Cotton 98%,1414.0,Polyester 21%
1,1,1,NaN,Cotton 98%,1415.0,Polyester 21%
2,2,2,NaN,Cotton 98%,1416.0,Polyester 21%
3,3,3,NaN,Cotton 98%,1417.0,Polyester 21%
4,4,4,NaN,Cotton 98%,1418.0,Polyester 21%


In [668]:
df_ref.shape

(1037, 6)

In [669]:
df_ref = df_ref.drop(columns=['level_0'], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated()]

In [670]:
df_ref.head()

,index,Spandex,cotton,polyester
0,0,NaN,Cotton 98%,Polyester 21%
1,1,NaN,Cotton 98%,Polyester 21%
2,2,NaN,Cotton 98%,Polyester 21%
3,3,NaN,Cotton 98%,Polyester 21%
4,4,NaN,Cotton 98%,Polyester 21%


In [671]:
df_ref.shape

(1037, 4)

Coluna Spandex

In [672]:
df1[1].unique()

array([' Spandex 2%', ' Spandex 1%', ' Polyester 21%', ' Polyester 20%',
       ' Polyester 19%', None, ' Polyester 6%'], dtype=object)

In [673]:
df_spandex = df1.loc[df1[1].str.contains('Spandex', na=True), 1]
df_spandex.name = 'spandex'

In [674]:
df_ref = pd.concat([df_ref.reset_index(), df_spandex.reset_index()], axis=1)
df_ref = df_ref.drop(columns=['Spandex'], axis=1)
df_ref.head()

,level_0,index,cotton,polyester,index,spandex
0,0,0,Cotton 98%,Polyester 21%,280.0,Spandex 2%
1,1,1,Cotton 98%,Polyester 21%,282.0,Spandex 2%
2,2,2,Cotton 98%,Polyester 21%,284.0,Spandex 2%
3,3,3,Cotton 98%,Polyester 21%,286.0,Spandex 2%
4,4,4,Cotton 98%,Polyester 21%,288.0,Spandex 2%


In [675]:
df_ref = df_ref.drop(columns=['level_0'], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated()]

In [676]:
df_ref.head()

,index,cotton,polyester,spandex
0,0,Cotton 98%,Polyester 21%,Spandex 2%
1,1,Cotton 98%,Polyester 21%,Spandex 2%
2,2,Cotton 98%,Polyester 21%,Spandex 2%
3,3,Cotton 98%,Polyester 21%,Spandex 2%
4,4,Cotton 98%,Polyester 21%,Spandex 2%


Data sample:

In [677]:
df_ref.sample(8)

,index,cotton,polyester,spandex
126,126,Cotton 98%,Polyester 21%,Spandex 2%
244,244,Cotton 98%,Polyester 21%,Spandex 2%
918,918,Cotton 93%,NaN,NaN
832,832,Cotton 77%,NaN,NaN
429,429,Cotton 98%,NaN,Spandex 2%
962,962,Cotton 98%,NaN,NaN
689,689,Cotton 80%,NaN,NaN
164,164,Cotton 98%,Polyester 19%,Spandex 2%


In [678]:
df_ref['cotton'].unique()

array(['Cotton 98%', 'Cotton 99%', 'Cotton 77%', 'Cotton 79%',
       'Cotton 80%', 'Cotton 78%', 'Cotton 100%', 'Cotton 93%'],
      dtype=object)

In [679]:
df_ref['polyester'].unique()

array([' Polyester 21%', ' Polyester 20%', ' Polyester 19%', None,
       ' Polyester 6%', nan], dtype=object)

In [680]:
df_ref['spandex'].unique()

array([' Spandex 2%', ' Spandex 1%', None, nan], dtype=object)

In [681]:
data.head()

,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition
280,280,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue_trashed,skinny_fit,"Cotton 98%, Spandex 2%"
282,282,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,denim_blue,skinny_fit,"Cotton 98%, Spandex 2%"
284,284,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed,skinny_fit,"Cotton 98%, Spandex 2%"
286,286,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue,skinny_fit,"Cotton 98%, Spandex 2%"
288,288,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed_out,skinny_fit,"Cotton 98%, Spandex 2%"


In [682]:
data.shape

(1037, 11)

In [683]:
df_ref.shape

(1037, 4)

In [684]:
data = pd.concat([data.reset_index(), df_ref.reset_index()], axis=1)
data.head()

,index,Unnamed: 0,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,Composition,level_0,index,cotton,polyester,spandex
0,280,280,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue_trashed,skinny_fit,"Cotton 98%, Spandex 2%",0,0,Cotton 98%,Polyester 21%,Spandex 2%
1,282,282,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,denim_blue,skinny_fit,"Cotton 98%, Spandex 2%",1,1,Cotton 98%,Polyester 21%,Spandex 2%
2,284,284,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed,skinny_fit,"Cotton 98%, Spandex 2%",2,2,Cotton 98%,Polyester 21%,Spandex 2%
3,286,286,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue,skinny_fit,"Cotton 98%, Spandex 2%",3,3,Cotton 98%,Polyester 21%,Spandex 2%
4,288,288,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed_out,skinny_fit,"Cotton 98%, Spandex 2%",4,4,Cotton 98%,Polyester 21%,Spandex 2%


In [685]:
data = data.drop(columns=['Unnamed: 0', 'level_0', 'Composition'], axis=1)
data = data.iloc[:, ~data.columns.duplicated()]
data.head()

,index,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,cotton,polyester,spandex
0,280,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue_trashed,skinny_fit,Cotton 98%,Polyester 21%,Spandex 2%
1,282,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,denim_blue,skinny_fit,Cotton 98%,Polyester 21%,Spandex 2%
2,284,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed,skinny_fit,Cotton 98%,Polyester 21%,Spandex 2%
3,286,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue,skinny_fit,Cotton 98%,Polyester 21%,Spandex 2%
4,288,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed_out,skinny_fit,Cotton 98%,Polyester 21%,Spandex 2%


In [686]:
data = data.drop(columns=['index'], axis=1)
data.head()

,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,cotton,polyester,spandex
0,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue_trashed,skinny_fit,Cotton 98%,Polyester 21%,Spandex 2%
1,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,denim_blue,skinny_fit,Cotton 98%,Polyester 21%,Spandex 2%
2,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed,skinny_fit,Cotton 98%,Polyester 21%,Spandex 2%
3,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue,skinny_fit,Cotton 98%,Polyester 21%,Spandex 2%
4,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed_out,skinny_fit,Cotton 98%,Polyester 21%,Spandex 2%


In [687]:
data.shape

(1037, 12)

Recuperar apenas os números da composição:

In [688]:
data['cotton'] = data['cotton'].apply(lambda x: int( re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['polyester'] = data['polyester'].apply(lambda x: int( re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
data['spandex'] = data['spandex'].apply(lambda x: int( re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

In [689]:
data.head()

,id,product_name,product_type,price,datetime,style_id,color_id,color,Fit,cotton,polyester,spandex
0,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue_trashed,skinny_fit,0.98,0.21,0.02
1,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,denim_blue,skinny_fit,0.98,0.21,0.02
2,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed,skinny_fit,0.98,0.21,0.02
3,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,light_denim_blue,skinny_fit,0.98,0.21,0.02
4,690449056,skinny_jeans,men_jeans_ripped,39.99,2022-01-24 14:33:19,690449,56,black_washed_out,skinny_fit,0.98,0.21,0.02
